In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
data = gpd.read_file('data/seoul_kais_data/Z_KAIS_TL_SPRD_MANAGE_11000.shp', encoding='euc-kr')
data_area = gpd.read_file('data/seoul_kais_data/Z_KAIS_TL_KODIS_BAS_11000.shp', encoding='euc-kr')

In [ ]:
plt.rcParams['figure.figsize']=(10,20)

#데이터들 지도로 띄어보기
data.plot()
plt.show()

data_area.plot()
plt.show()

In [ ]:
data.head()

In [ ]:
preprocessingRoadData = []

for area in (data_area['geometry'])[0:2]:
    roadList = data[area.crosses(data['geometry'])]
    roadAmount = len(roadList)
    #print(roadList.head())

    preprocessingRoadList = []
    roadWidth = 0
    for roadIndex in roadList.index.to_list():
        roadWidth = roadWidth + (roadList.loc[roadIndex])['ROAD_BT']
        print((roadList.loc[roadIndex])['ROAD_LT'])

        

        if (roadList.loc[roadIndex])['ROAD_LT'] < 2000:
            preprocessingRoadList.append(roadList.loc[roadIndex])
        
    
    roadWidth = roadWidth / roadAmount

    roadList.plot()
    plt.show()
    


    if 22 >= len(preprocessingRoadList) >= 15:
        preprocessingRoadData.append(preprocessingRoadList)


print("총 데이터 개수 : ", len(preprocessingRoadData))
    

In [ ]:
road = data[data['RN']=='이태원로']
road.plot()

In [ ]:
def isContainPeopleInRoad(peoplePos, roadData):
    indexList = roadData.index.to_list()
    for roadIndex in indexList: #지도를 구성하는 도로 하나하나 확인
        roadWidth = roadData.loc[roadIndex]['ROAD_BT']

        posList = list((roadData.loc[roadIndex]['geometry']).coords)
        for posIndex in range(0, len(posList)): #도로의 좌표들을 확인
            if posList[posIndex] == posList[len(posList)-1]: #마지막 좌표는 확인하지 않음
                break
            
            firstPos = posList[posIndex]
            lastPos = posList[posIndex+1]
            

            #첫번째와 마지막 좌표와 도로폭으로 공간을 계산한 후 사람의 위치가 포함되는지 확인
            if firstPos[0] == lastPos[0]: #x좌표가 같은 경우
                if firstPos[0] - roadWidth/2 <= peoplePos[0] <= firstPos[0] + roadWidth/2:
                    if firstPos[1] <= peoplePos[1] <= lastPos[1] or firstPos[1] >= peoplePos[1] >= lastPos[1]:
                        return True
            elif firstPos[1] == lastPos[1]: #y좌표가 같은 경우
                if firstPos[1] - roadWidth/2 <= peoplePos[1] <= firstPos[1] + roadWidth/2:
                    if firstPos[0] <= peoplePos[0] <= lastPos[0] or firstPos[0] >= peoplePos[0] >= lastPos[0]:
                        return True
            else: #x좌표와 y좌표가 다른 경우
                if firstPos[0] - roadWidth/2 <= peoplePos[0] <= firstPos[0] + roadWidth/2:
                    if firstPos[1] - roadWidth/2 <= peoplePos[1] <= firstPos[1] + roadWidth/2:
                        return True
    
    return False